In [56]:
import os
import re
import codecs

mmode0 = re.compile(r'\$\$.+?\$\$', flags = re.DOTALL | re.UNICODE)
mmode1 = re.compile(r'\$.+?\$', flags = re.DOTALL | re.UNICODE)
mmode2 = re.compile(r'\\\[.+?\\\]', flags = re.DOTALL | re.UNICODE)
mmode3 = re.compile(r'\\begin{equation}.+?\\end{equation}', flags = re.DOTALL | re.UNICODE)
mmode4 = re.compile(r'\\begin{eqnarray}.+?\\end{eqnarray}', flags = re.DOTALL | re.UNICODE)
mmode5 = re.compile(r'\\begin{equation\*}.+?\\end{equation\*}', flags = re.DOTALL | re.UNICODE)
mmode6 = re.compile(r'\\begin{eqnarray\*}.+?\\end{eqnarray\*}', flags = re.DOTALL | re.UNICODE)

In [46]:
def primary_processing(inp):
    inp = re.sub(r'.*\\newcommand.*', '', inp)
    
    s0 = mmode0.findall(inp)
    inp = mmode0.sub('', inp)

    s1 = mmode1.findall(inp)
    inp = mmode1.sub('', inp)

    s2 = mmode2.findall(inp)
    inp = mmode2.sub('', inp)

    s3 = mmode3.findall(inp)
    inp = mmode3.sub('', inp)

    s4 = mmode4.findall(inp)
    inp = mmode4.sub('', inp)
    
    s5 = mmode5.findall(inp)
    inp = mmode5.sub('', inp)
    
    s6 = mmode6.findall(inp)
    inp = mmode6.sub('', inp)

#     print s0[0:5]
#     print s1[0:5]
#     print s2[0:5]
#     print s3[0:5]
#     print s4[0:5]

    ret = []
    
    [ret.append(i) for i in s0]
    [ret.append(i) for i in s1]
    [ret.append(i) for i in s2]
    [ret.append(i) for i in s3]
    [ret.append(i) for i in s4]
    [ret.append(i) for i in s5]
    [ret.append(i) for i in s6]
    
    return ret

In [47]:
def secondary_processing(inp):
    inp = inp.replace('\n',' ')
    
    if mmode0.match(inp) != None:
        inp = inp.replace('$$','')
    
    if mmode1.match(inp) != None:
        inp = inp.replace('$','')
    
    if mmode2.match(inp) != None:
        inp = inp.replace('\\[','')
        inp = inp.replace('\\]','')
    
    if mmode3.match(inp) != None:
        inp = inp.replace('\\begin{equation}','')
        inp = inp.replace('\\end{equation}','')
    
    if mmode4.match(inp) != None:
        inp = inp.replace('\\begin{eqnarray}','')
        inp = inp.replace('\\end{eqnarray}','')
        
    if mmode5.match(inp) != None:
        inp = inp.replace('\\begin{equation*}','')
        inp = inp.replace('\\end{equation*}','')
    
    if mmode6.match(inp) != None:
        inp = inp.replace('\\begin{eqnarray*}','')
        inp = inp.replace('\\end{eqnarray*}','')
    
    return inp

In [48]:
def tertiary_processing(inp):
    inp = inp.replace('\\nonumber', '')
    inp = inp.replace('&',' ')
    
    inp = inp.split('\\\\')
    ret = []
    for i in inp:
        if i != '\\':
            ret.append(i.strip())
    
    return ret

In [49]:
def full_processing(inp):
    inp = primary_processing(inp)
    inp = [secondary_processing(i) for i in inp]
    
    processed = []
    [processed.extend(tertiary_processing(i)) for i in inp]
    processed = [i for i in processed if len(i) <= 600]
    
    return processed

In [50]:
text = file('../../Dataset/2000/0001081','r').read().decode('cp1252')
# out = primary_processing(text)

# # out = [secondary_processing(i) for i in out]

# # for o in out:
# #     print o

# # print mmode4.match(out[-1]).group(0)
# print out[-1]
# print '#'
# print secondary_processing(out[-1])
try:
    out = full_processing(text)
except UnicodeDecodeError as obj:
    print obj
    raise
# print out
for o in out:
    print '#'
    print o
    print '#'

#
\mbox{\rule[-6pt]{0.4pt}{14pt}}_{\,#2}
#
#
\ZQ
#
#
\Q
#
#
1:1
#
#
\ZQ
#
#
\manN_0
#
#
\Sigma
#
#
\ZQ
#
#
\manN
#
#
\Q
#
#
\Q^2=0
#
#
\Q
#
#
\Omega
#
#
S
#
#
\ZQ\subset\manN
#
#
\Q=\{\Omega,\cdot\,\}
#
#
\Q=(S,\cdot\,)
#
#
\manN
#
#
\ZQ
#
#
\Q
#
#
1:1
#
#
\ZQ
#
#
\manN
#
#
\ZQ
#
#
\ZQ
#
#
\manN_0
#
#
\Sigma
#
#
\manN
#
#
\ZQ
#
#
\manN
#
#
\Q
#
#
(\manN_0,\Sigma)
#
#
(\manN,\ZQ)
#
#
\Q
#
#
\aA
#
#
\wwedge\aA
#
#
\S\aA
#
#
L_\infty
#
#
Sp(2)
#
#
\aA
#
#
(\aA,\aA^*,\aA \oplus \aA^*)
#
#
\QP{}
#
#
\manN
#
#
\gerst{\cdot{}}{\cdot{}}
#
#
F,G,H \in \func{\manN}
#
#
\manN
#
#
\Q
#
#
\Q^2=0
#
#
\p(\cdot)
#
#
\kappa=0
#
#
\kappa=1
#
#
\QP{}
#
#
\Q
#
#
\ZQ
#
#
\Q
#
#
\QP{}
#
#
\manN
#
#
\ZQ
#
#
\ideal{\ZQ}\subset \func\manN
#
#
\ZQ
#
#
\Q
#
#
f\in\ideal{\ZQ}
#
#
f_\alpha,\,\Gamma^\alpha\in\func\manN
#
#
\Q
#
#
\ideal{\ZQ}
#
#
\manL\subset\manN
#
#
\Q
#
#
\ZQ
#
#
\manN
#
#
f,g \in \func\manN
#
#
\ZQ
#
#
\Q
#
#
\gerst{f_\alpha     (\Q\Gamma^\alpha)}{(\Q\Gamma^\beta) g_\beta}|_{\ZQ}=   (f_\alpha\ge

In [58]:
def main():
    out = codecs.open('../Data/Formulae', 'w', 'cp1252')
    metaout = codecs.open('../Data/Meta', 'w', 'cp1252')
    
    for year in xrange(1992, 2004):
        
        print 'Processing year {0}'.format(year)
        files = os.listdir('../../Dataset/{0}'.format(year))
        
        for afile in files:
            try:
                text = open('../../Dataset/{0}/{1}'.format(year, afile), 'r').read().decode('cp1252', errors='ignore')
                for form in full_processing(text):
                    out.write(u'{0}\n'.format(form))
                    metaout.write(u'{0} {1}\n'.format(year, afile))
            except Exception as obj:
                print year, afile
                print obj
                raise

                
    out.close()
    metaout.close()
        
if __name__ == '__main__':
    main()

Processing year 1992
Processing year 1993
Processing year 1994
Processing year 1995


KeyboardInterrupt: 